<a href="https://colab.research.google.com/github/somyamangla/ensemble/blob/main/Copy_of_Ensemble_Cervical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, precision_score, accuracy_score, recall_score, balanced_accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

In [ ]:
df=pd.read_csv('CERVICAL FEATURES SELECTED DATA.csv')

In [ ]:
df.head()

,Hormonal Contraceptives,Hormonal Contraceptives (years),Dx,Dx:Cancer,IUD,Dx:HPV,IUD (years),Smokes (packs/year),STDs:vulvo-perineal condylomatosis,STDs:condylomatosis,Biopsy
0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0
3,1,3,0,1,0,1,0,37,0,0,0
4,1,15,0,0,0,0,0,0,0,0,0


In [ ]:
for x in df.columns:
    df[x].replace("?",np.nan,inplace=True)

def imputation_null_value(df):
    for x in  df.columns:
        value =df[x].mode(dropna=True).loc[0]
        df[x].fillna(value,inplace=True)
imputation_null_value(df)

In [ ]:
df.isnull().sum()

Hormonal Contraceptives               0
Hormonal Contraceptives (years)       0
Dx                                    0
Dx:Cancer                             0
IUD                                   0
Dx:HPV                                0
IUD (years)                           0
Smokes (packs/year)                   0
STDs:vulvo-perineal condylomatosis    0
STDs:condylomatosis                   0
Biopsy                                0
dtype: int64

In [ ]:
X = df.drop(columns = ['Biopsy'])
y = df['Biopsy']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [ ]:
#create new a knn model
knn = KNeighborsClassifier()
#create a dictionary of all values we want to test for n_neighbors
params_knn = {'n_neighbors': np.arange(1, 25)}
#use gridsearch to test all values for n_neighbors
knn_gs = GridSearchCV(knn, params_knn, cv=5)
#fit model to training data
knn_gs.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])})

In [ ]:
#save best model
knn_best = knn_gs.best_estimator_

#check best n_neigbors value
print(knn_gs.best_params_)

{'n_neighbors': 4}


In [ ]:
#create a new rf classifier
rf = RandomForestClassifier()

#create a dictionary of all values we want to test for n_estimators
params_rf = {'n_estimators': [50, 100, 200]}

#use gridsearch to test all values for n_estimators
rf_gs = GridSearchCV(rf, params_rf, cv=5)

#fit model to training data
rf_gs.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'n_estimators': [50, 100, 200]})

In [ ]:
#save best model
rf_best = rf_gs.best_estimator_

#check best n_estimators value
print(rf_gs.best_params_)

{'n_estimators': 200}


In [ ]:
#create a new logistic regression model
log_reg = LogisticRegression()

#fit the model to the training data
log_reg.fit(X_train, y_train)

LogisticRegression()

In [ ]:
tuned_params = {'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000], 'penalty': ['l1', 'l2']}
log_reg = GridSearchCV(LogisticRegression(solver='liblinear'), tuned_params, scoring='roc_auc', n_jobs=-1)
log_reg.fit(X_train, y_train)

GridSearchCV(estimator=LogisticRegression(solver='liblinear'), n_jobs=-1,
             param_grid={'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000,
                               10000],
                         'penalty': ['l1', 'l2']},
             scoring='roc_auc')

In [ ]:

print('knn: {}'.format(knn_best.score(X_test, y_test)))
print('rf: {}'.format(rf_best.score(X_test, y_test)))
print('log_reg: {}'.format(log_reg.score(X_test, y_test)))

knn: 0.936046511627907
rf: 0.936046511627907
log_reg: 0.7001693958215698


In [ ]:
from sklearn.ensemble import VotingClassifier

In [ ]:
#create a dictionary of our models
estimators=[('knn', knn_best), ('rf', rf_best), ('log_reg', log_reg)]

#create our voting classifier, inputting our models
ensemble = VotingClassifier(estimators, voting='hard')

#fit model to training data
ensemble.fit(X_train, y_train)

#test our model on the test data
ensemble.score(X_test, y_test)

0.936046511627907